In [1]:
import eyed3
import glob
import wx

%gui wx

In [3]:
class MP3TagEditor(wx.Frame):

    def __init__(self, *args, **kwargs):
        super().__init__(parent=None, title="MP3 Tag Editor", 
                         *args, **kwargs)

        self.SetSize((450, 600))

        self.panel = MainPanel(self)
        self.create_menu()

    def create_menu(self):
        menu_bar = wx.MenuBar()
        
        file_menu = wx.Menu()
        open_folder_item = file_menu.Append(
            wx.ID_ANY, 'Open Folder', 'Open a folder with MP3 files'
        )
        menu_bar.Append(file_menu, '&File')
        self.Bind(
            event=wx.EVT_MENU,
            handler=self.on_open_folder,
            source=open_folder_item
        )
        self.SetMenuBar(menu_bar)

    def on_open_folder(self, event):
        title = "Choose a folder:"
        dialog = wx.DirDialog(self, title, style=wx.DD_DEFAULT_STYLE)
        if dialog.ShowModal() == wx.ID_OK:
            self.panel.update_listings(dialog.GetPath())
        dialog.Destroy()


class MainPanel(wx.Panel):

    def __init__(self, parent):
        super().__init__(parent)

        sizer = wx.BoxSizer(wx.VERTICAL)
        self.row_obj_dict = {}

        self.list_ctrl = wx.ListCtrl(
            self, 
            size=(-1, 500),
            style=wx.LC_REPORT | wx.BORDER_SUNKEN
        )
        self.list_ctrl.InsertColumn(0, 'Artist', width=140)
        self.list_ctrl.InsertColumn(1, 'Title', width=140)
        self.list_ctrl.InsertColumn(2, 'Album', width=140)
        sizer.Add(self.list_ctrl, 0, wx.ALL | wx.EXPAND, 5)

        edit_button = wx.Button(self, label='Edit')
        edit_button.Bind(wx.EVT_BUTTON, self.on_edit)
        sizer.Add(edit_button, 0, wx.ALL | wx.CENTER, 5)
        self.SetSizer(sizer)

    def on_edit(self, event):
        selection = self.list_ctrl.GetFocusedItem()
        if selection >= 0:
            mp3 = self.row_obj_dict[selection]
            dialog = EditDialog(mp3)
            dialog.ShowModal()
            self.update_listings(self.current_folder)
            dialog.Destroy()

    def update_listings(self, folder_path):
        self.current_folder = folder_path
        self.list_ctrl.ClearAll()

        self.list_ctrl.InsertColumn(0, 'Artist', width=140)
        self.list_ctrl.InsertColumn(1, 'Album', width=140)
        self.list_ctrl.InsertColumn(2, 'Title', width=140)
        self.list_ctrl.InsertColumn(3, 'Year', width=140)

        mp3s = glob.glob(folder_path + '/*.mp3')
        mp3_objects = []
        for i, mp3 in enumerate(mp3s):
            mp3 = eyed3.load(mp3)
            self.list_ctrl.InsertItem(i, mp3.tag.artist)
            self.list_ctrl.SetItem(i, 1, mp3.tag.album)
            self.list_ctrl.SetItem(i, 2, mp3.tag.title)
            self.row_obj_dict[i] = mp3

class EditDialog(wx.Dialog):
    
    def __init__(self, mp3):
        title = f"Editing '{mp3.tag.title}'"
        super().__init__(parent=None, title=title, size=(400, 200))
        self.mp3 =  mp3
        self.sizer = wx.BoxSizer(wx.VERTICAL)

        self.artist = wx.TextCtrl(self, value=mp3.tag.artist)
        self.add_widgets('Artist', self.artist)

        self.album = wx.TextCtrl(self, value=mp3.tag.album)
        self.add_widgets('Album', self.album)

        self.title = wx.TextCtrl(self, value=mp3.tag.title)
        self.add_widgets('Title', self.title)

        btn_sizer = wx.BoxSizer()
        save_btn = wx.Button(self, label='Save')
        save_btn.Bind(wx.EVT_BUTTON, self.on_save)
        btn_sizer.Add(save_btn, 0, wx.ALL | wx.CENTER, 5)
        btn_sizer.Add(
            wx.Button(self, id=wx.ID_CANCEL), 0, wx.ALL | wx.CENTER, 5
        )
        self.sizer.Add(btn_sizer, 0, wx.ALL | wx.CENTER, 5)
        self.SetSizer(self.sizer)

    def add_widgets(self, label_text, text_ctrl):
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        label = wx.StaticText(self, label=label_text, size=(70, -1))

        row_sizer.Add(label, 0, wx.ALL | wx.CENTER, 5)
        row_sizer.Add(text_ctrl, 0, wx.ALL | wx.CENTER, 5)
        self.sizer.Add(row_sizer, 0, wx.ALL | wx.CENTER, 5)

    def on_save(self, event):
        self.mp3.tag.artist = self.artist.GetValue()
        self.mp3.tag.album = self.album.GetValue()
        self.mp3.tag.title = self.title.GetValue()
        self.mp3.tag.save()
        self.Close()

In [4]:
wx.GetApp();
MP3TagEditor().Show()
# frame.Show()

True